## Load data

In [82]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from scipy.stats import chi2_contingency, kruskal
import os
import networkx as nx
import warnings
warnings.filterwarnings("ignore")
import matplotlib.patches as mpatches

In [83]:
base_path = '../data/full'
algo = 'reverse_hybrid'
patient_df = pd.read_csv('../../data/thesis/cll_broad_2022_clinical_data_thesis.csv')
mutation_df = pd.read_csv('../../data/thesis/cll_broad_2022_mutations_thesis.csv')
community_df = pd.read_csv(os.path.join(base_path, algo, 'community_assignments.csv'))

In [84]:
from collections import defaultdict
comm_map = defaultdict(list)
for _, row in community_df.iterrows():
    comm_map[row['patientId']].append(row['communityId'])
def resolve_community(comms):
    return min(comms) if comms else np.nan
patient_df['communityId'] = patient_df['patientId'].map(lambda pid: resolve_community(comm_map.get(pid, [])))
patient_df['communityId'] = patient_df['communityId'].astype('Int64')
print(patient_df[['patientId', 'communityId']].head())
print(patient_df['communityId'].isna().sum())

    patientId  communityId
0  P-CRC-0001            2
1  P-CRC-0002            2
2  P-CRC-0003            0
3  P-CRC-0004            3
4  P-CRC-0005            3
135


In [85]:
columns = [
    'patientId', 'CLL_EPITYPE', 'MUTATION_COUNT', 'TUMOR_MOLECULAR_SUBTYPE',
    'AGE_SAMPLING', 'COHORT', 'IGHV_MUTATION_STATUS', 'SEX',
    'TREATMENT_AFTER_SAMPLING', 'communityId'
]
patient_df = patient_df[columns]
patient_df.head()

,patientId,CLL_EPITYPE,MUTATION_COUNT,TUMOR_MOLECULAR_SUBTYPE,AGE_SAMPLING,COHORT,IGHV_MUTATION_STATUS,SEX,TREATMENT_AFTER_SAMPLING,communityId
0,P-CRC-0001,n-CLL,26.0,U-CLL,46.0,UCSD,unmutated,Female,Chemo + Ab,2
1,P-CRC-0002,n-CLL,23.0,U-CLL,56.0,UCSD,unmutated,Male,Chemo + Ab,2
2,P-CRC-0003,n-CLL,7.0,U-CLL,63.0,UCSD,unmutated,Female,Chemo + Ab,0
3,P-CRC-0004,m-CLL,30.0,M-CLL,51.0,UCSD,mutated,Male,Chemo + Ab,3
4,P-CRC-0005,n-CLL,23.0,U-CLL,37.0,UCSD,unmutated,Male,Chemo + Ab,3


In [86]:
patient_df.isna().sum()

patientId                     0
CLL_EPITYPE                   0
MUTATION_COUNT              161
TUMOR_MOLECULAR_SUBTYPE     160
AGE_SAMPLING                  3
COHORT                        0
IGHV_MUTATION_STATUS         23
SEX                           3
TREATMENT_AFTER_SAMPLING    747
communityId                 135
dtype: int64

In [87]:
patient_df['communityId'].value_counts().sort_index()

communityId
0    156
1     50
2     80
3     29
4     37
5    115
6     57
7    110
8    226
9    148
Name: count, dtype: Int64

## Mutations

In [88]:
top_mutations_set = {'TTN', 'CSMD1', 'PCLO', 'CSMD3', 'KRAS', 'FAT1', 'EGR2', 'BRAF', 'ATM', 'SF3B1', 'MYD88', 'CHD2', 'POT1', 'MUC16', 'ZMYM3', 'AHNAK', 'ZFHX4', 'TP53', 'LRP1B', 'KLHL6', 'XPO1', 'USH2A', 'NOTCH1'}
mutation_df['has_mutation'] = True
mutation_pivot = mutation_df[mutation_df['hugoGeneSymbol'].isin(top_mutations_set)].pivot_table(index='patientId', columns='hugoGeneSymbol', values='has_mutation', fill_value=0.0)
for gene in top_mutations_set:
    if gene not in mutation_pivot.columns:
        mutation_pivot[gene] = 0.0
mutation_pivot = mutation_pivot[list(top_mutations_set)]
patient_df = patient_df.merge(mutation_pivot, how='left', left_on='patientId', right_index=True)
patient_df[list(top_mutations_set)] = patient_df[list(top_mutations_set)].fillna(0.0)
display(patient_df.head())
print(patient_df.shape)
print(patient_df.columns)

,patientId,CLL_EPITYPE,MUTATION_COUNT,TUMOR_MOLECULAR_SUBTYPE,AGE_SAMPLING,COHORT,IGHV_MUTATION_STATUS,SEX,TREATMENT_AFTER_SAMPLING,communityId,...,MUC16,LRP1B,USH2A,CSMD3,EGR2,SF3B1,POT1,ZFHX4,BRAF,ZMYM3
0,P-CRC-0001,n-CLL,26.0,U-CLL,46.0,UCSD,unmutated,Female,Chemo + Ab,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,P-CRC-0002,n-CLL,23.0,U-CLL,56.0,UCSD,unmutated,Male,Chemo + Ab,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P-CRC-0003,n-CLL,7.0,U-CLL,63.0,UCSD,unmutated,Female,Chemo + Ab,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,P-CRC-0004,m-CLL,30.0,M-CLL,51.0,UCSD,mutated,Male,Chemo + Ab,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,P-CRC-0005,n-CLL,23.0,U-CLL,37.0,UCSD,unmutated,Male,Chemo + Ab,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(1143, 33)
Index(['patientId', 'CLL_EPITYPE', 'MUTATION_COUNT', 'TUMOR_MOLECULAR_SUBTYPE',
       'AGE_SAMPLING', 'COHORT', 'IGHV_MUTATION_STATUS', 'SEX',
       'TREATMENT_AFTER_SAMPLING', 'communityId', 'XPO1', 'NOTCH1', 'AHNAK',
       'TP53', 'CHD2', 'PCLO', 'TTN', 'MYD88', 'CSMD1', 'KLHL6', 'ATM', 'KRAS',
       'FAT1', 'MUC16', 'LRP1B', 'USH2A', 'CSMD3', 'EGR2', 'SF3B1', 'POT1',
       'ZFHX4', 'BRAF', 'ZMYM3'],
      dtype='object')


In [89]:
patient_df.isna().sum()

patientId                     0
CLL_EPITYPE                   0
MUTATION_COUNT              161
TUMOR_MOLECULAR_SUBTYPE     160
AGE_SAMPLING                  3
COHORT                        0
IGHV_MUTATION_STATUS         23
SEX                           3
TREATMENT_AFTER_SAMPLING    747
communityId                 135
XPO1                          0
NOTCH1                        0
AHNAK                         0
TP53                          0
CHD2                          0
PCLO                          0
TTN                           0
MYD88                         0
CSMD1                         0
KLHL6                         0
ATM                           0
KRAS                          0
FAT1                          0
MUC16                         0
LRP1B                         0
USH2A                         0
CSMD3                         0
EGR2                          0
SF3B1                         0
POT1                          0
ZFHX4                         0
BRAF    

## Decision Tree

In [90]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
df = patient_df.copy(deep=True)
features = ['AGE_SAMPLING', 'IGHV_MUTATION_STATUS', 'CLL_EPITYPE', 'TUMOR_MOLECULAR_SUBTYPE', 'MUTATION_COUNT', 'SEX'] + list(top_mutations_set)
target = 'communityId'
df = df.dropna(subset=[target])
print(f'Number of rows after dropping missing target: {len(df)}')
category_orders = {
    'IGHV_MUTATION_STATUS': sorted(df['IGHV_MUTATION_STATUS'].dropna().unique()),
    'CLL_EPITYPE': sorted(df['CLL_EPITYPE'].dropna().unique()),
    'TUMOR_MOLECULAR_SUBTYPE': sorted(df['TUMOR_MOLECULAR_SUBTYPE'].dropna().unique()),
    'SEX': ['Female', 'Male']
}
for col, order in category_orders.items():
    df[col] = df[col].fillna('Unknown')
category_orders = {
    'IGHV_MUTATION_STATUS': sorted(df['IGHV_MUTATION_STATUS'].dropna().unique()),
    'CLL_EPITYPE': sorted(df['CLL_EPITYPE'].dropna().unique()),
    'TUMOR_MOLECULAR_SUBTYPE': sorted(df['TUMOR_MOLECULAR_SUBTYPE'].dropna().unique()),
    'SEX': ['Female', 'Male']
}
for col in features:
    if col not in category_orders:
        df[col] = df[col].fillna(df[col].median())
ordinal_encoder = OrdinalEncoder(categories=[category_orders[col] for col in category_orders])
cat_features = list(category_orders.keys())
df[cat_features] = ordinal_encoder.fit_transform(df[cat_features])
X = df[features]
y = df[target].astype(int)
print(f'Feature matrix shape: {X.shape}, Target vector shape: {y.shape}')
clf = DecisionTreeClassifier(max_depth=100, random_state=42)
clf.fit(X, y)
# plt.figure(figsize=(24, 12))
# plot_tree(clf, feature_names=features, class_names=[str(i) for i in sorted(y.unique())], filled=True, fontsize=6, rounded=True)
# plt.title('Decision Tree Visualization for Leiden Communities', fontsize=16)
# plt.savefig('../../figures/leiden_decision_tree.png', dpi=300, bbox_inches='tight')
# plt.show()
# plt.close()
# tree_rules = export_text(clf, feature_names=features)
# print(tree_rules)

Number of rows after dropping missing target: 1008
Feature matrix shape: (1008, 29), Target vector shape: (1008,)


DecisionTreeClassifier(max_depth=100, random_state=42)

In [91]:
from sklearn.model_selection import cross_val_score
depths = range(100, 10, -1)
cv_scores = []
for depth in depths:
    clf = DecisionTreeClassifier(max_depth=depth, random_state=42)
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())
best_depth = depths[np.argmax(cv_scores)]
print(f'Best max depth: {best_depth}')
print(f'Best score: {max(cv_scores):.2f}')

Best max depth: 11
Best score: 0.22


In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier(max_depth=best_depth, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f'Test Accuracy: {accuracy_score(y_test, y_pred):.2f}')
print(f'Test Precision: {precision_score(y_test, y_pred, average="weighted", zero_division=0):.2f}')
print(f'Test Recall: {recall_score(y_test, y_pred, average="weighted", zero_division=0):.2f}')
print(f'Test F1 Score: {f1_score(y_test, y_pred, average="weighted", zero_division=0):.2f}')

Test Accuracy: 0.18
Test Precision: 0.16
Test Recall: 0.18
Test F1 Score: 0.16


## Association Rule Mining

In [93]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Assume patient_df is already loaded
dfdf = patient_df.copy(deep=True)

dfdf = dfdf.dropna(subset=['communityId'])
# Convert communityId to string
dfdf['communityId'] = dfdf['communityId'].astype(str)
# dfdf = dfdf[dfdf['communityId'].isin(['3', '4'])]

# Step 1: Replace missing values with 'Unknown' for categorical columns
categorical_cols = [
    'IGHV_MUTATION_STATUS', 
    'CLL_EPITYPE', 
    'TUMOR_MOLECULAR_SUBTYPE', 
    'SEX', 
    # 'TREATMENT_AFTER_SAMPLING',
    'communityId',
    # 'SF3B1',
    # 'TP53',
    #                 'POT1', 'CSMD1', 'ZMYM3', 'PCLO',  'MYD88', 'NOTCH1',  'ATM', 'TTN',
    #                 'KRAS', 'BRAF', 'CHD2', 'XPO1', 'ZFHX4', 'EGR2', 'USH2A', 'FAT1', 'KLHL6', 'AHNAK', 'LRP1B', 'CSMD3', 'MUC16'
                
    'SF3B1',
    'TP53',
    'PCLO',  
    'NOTCH1',  
    'ATM',
    'ZFHX4',
    'FAT1', 
    'KLHL6'         
    ]
for col in categorical_cols:
    dfdf[col] = dfdf[col].fillna('Unknown')

# Remove 'unclassified' from CLL_EPITYPE if needed
dfdf['CLL_EPITYPE'] = dfdf['CLL_EPITYPE'].replace('unclassified', 'Unknown')

# Step 2: Bin numerical features with 'Unknown' for missing values
def bin_age(age):
    if pd.isna(age):
        return 'Unknown'
    elif age <= 60:
        return 'Young'
    elif age <= 70:
        return 'Middle'
    else:
        return 'Old'
    #[0, 60, 64, 70, 120]
    # elif age <= 60:
    #     return 'Young'  
    # elif age <= 64:
    #     return 'Middle'
    # elif age <= 70:
    #     return 'Old'
    # else:
    #     return 'Very Old'

def bin_mutation_count(count):
    if pd.isna(count):
        return 'Unknown'
    elif count <= 20:
        return 'Low'
    elif count <= 30:
        return 'Medium'
    else:
        return 'High'
    # [0, 23, 25, 30, 500]
    # elif count <= 23:
    #     return 'Low'
    # elif count <= 25:
    #     return 'Medium'
    # elif count <= 30:
    #     return 'High'
    # else:
    #     return 'Very High'

dfdf['AGE_BIN'] = dfdf['AGE_SAMPLING'].apply(bin_age)
dfdf['MUTATION_BIN'] = dfdf['MUTATION_COUNT'].apply(bin_mutation_count)

# Step 3: Encode categorical variables into transaction format
transactions = dfdf[
    ['AGE_BIN', 'MUTATION_BIN'] + 
    categorical_cols
    ]
og_transactions = transactions.copy(deep=True)

# Convert to one-hot encoding
encoded_df = pd.get_dummies(transactions)

# Step 4: Apply Apriori algorithm
frequent_itemsets = apriori(encoded_df, min_support=0.01, use_colnames=True)
print(f"Number of frequent itemsets: {len(frequent_itemsets)}")

# Step 5: Generate association rules with communityId as consequent
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rules = rules[rules['consequents'].apply(lambda x: any('communityId_' in item for item in x))]

# in rules, can support to converted to #records instead of percentage
# Convert support from percentage to number of records
rules['support'] = rules['support'] * len(dfdf)

# Step 6: Sort and display top rules
top_rules = rules.sort_values(by=[ 'support',  'confidence', 'lift'], ascending=False)
display(top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(50))

# top_rules[['antecedents', 'consequents', 'support', 'confidence']].to_csv("./rules.csv")

Number of frequent itemsets: 3363


,antecedents,consequents,support,confidence,lift
1912,"(SEX_Female, MUTATION_BIN_Unknown, IGHV_MUTATI...",(communityId_8),13.0,0.619048,2.761062
2625,"(SEX_Female, IGHV_MUTATION_STATUS_mutated, AGE...",(communityId_8),11.0,0.687500,3.066372


In [94]:
import os

def save_rules(rules, index):
    # Keep only rules where the consequent is exactly {'communityId_X'}
    target_consequent = {f'communityId_{index}'}
    top_rules = rules[rules['consequents'].apply(lambda x: x == target_consequent)]
    print(f"Number of rules for community {index}: {len(top_rules)}")
    if top_rules.empty:
        return

    # Remove rules with 'Unknown' in antecedents or consequents
    def has_unknown(itemset):
        return any('Unknown' in item for item in itemset)

    top_rules = top_rules[~top_rules['antecedents'].apply(has_unknown)]
    top_rules = top_rules[~top_rules['consequents'].apply(has_unknown)]

    print(f"Number of rules after removing 'Unknown': {len(top_rules)}")
    if top_rules.empty:
        return

    # Initialize redundancy flags
    is_redundant = [False] * len(top_rules)
    subset_ids = [None] * len(top_rules)

    antecedents_list = top_rules['antecedents'].tolist()

    for i, ant_i in enumerate(antecedents_list):
        for j, ant_j in enumerate(antecedents_list):
            if i != j and ant_j.issubset(ant_i):
                is_redundant[i] = True
                subset_ids[i] = top_rules.index[j]
                break

    # Add redundancy information to DataFrame
    top_rules = top_rules.copy()
    top_rules['isRedundant'] = is_redundant
    top_rules['subsetId'] = subset_ids

    # top_rules = top_rules[top_rules['isRedundant'] == False]

    # Sort by confidence and lift
    top_rules = top_rules.sort_values(by=['confidence', 'lift'], ascending=False)

    # Display top rules
    display(top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'isRedundant', 'subsetId']].head(30))

    # Save to CSV
    top_rules_path = "../data/rules/"
    os.makedirs(top_rules_path, exist_ok=True)
    top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'isRedundant', 'subsetId']].to_csv(
        f"{top_rules_path}/leiden_rules_{index}.csv"
    )



In [95]:
for i in range(10):
    save_rules(rules, i)

Number of rules for community 0: 0
Number of rules for community 1: 0
Number of rules for community 2: 0
Number of rules for community 3: 0
Number of rules for community 4: 0
Number of rules for community 5: 0
Number of rules for community 6: 0
Number of rules for community 7: 0
Number of rules for community 8: 2
Number of rules after removing 'Unknown': 0
Number of rules for community 9: 0


## Categorical Features - NMI/ARI Quality Metrics Table

In [96]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
df = patient_df.copy(deep=True)
features = ['IGHV_MUTATION_STATUS', 'CLL_EPITYPE', 'TUMOR_MOLECULAR_SUBTYPE', 'COHORT', 'SEX']
df = df.dropna(subset=['communityId'])
results = {'Feature': [], 'NMI': [], 'ARI': []}
for feature in features:
    valid_rows = df[[feature, 'communityId']].dropna()
    x = valid_rows[feature]
    y = valid_rows['communityId'].astype(int)
    if x.dtype == 'object': x = pd.factorize(x)[0]
    nmi = normalized_mutual_info_score(y, x)
    ari = adjusted_rand_score(y, x)
    results['Feature'].append(feature)
    results['NMI'].append(round(nmi, 4))
    results['ARI'].append(round(ari, 4))
metrics_df = pd.DataFrame(results)
metrics_df.set_index('Feature', inplace=True)
latex_table = metrics_df.to_latex(index=True, float_format='%.4f', column_format='|l|c|c|')
metrics_df

,NMI,ARI
Feature,,
IGHV_MUTATION_STATUS,0.0655,0.0621
CLL_EPITYPE,0.0649,0.0568
TUMOR_MOLECULAR_SUBTYPE,0.0064,0.0012
COHORT,0.0817,0.0333
SEX,0.0102,-0.0022
